In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import pipeline


In [2]:
all_emails = pd.read_csv("all_emails.csv")
all_emails["Date"] = all_emails["Date"].astype("datetime64[ns]")
all_emails["Subject"] = all_emails["Subject"].fillna("")
all_emails["Body_Quoted"] = all_emails["Body_Quoted"].fillna("")


Time for first 10k subjects:

- GPU: 58.7 sec
- CPU: 2 min 45 sec

https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion

https://stackoverflow.com/questions/64320883/the-size-of-tensor-a-707-must-match-the-size-of-tensor-b-512-at-non-singleto


In [3]:
classifier = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    top_k=None,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)


In [4]:
%%time
prediction = classifier(all_emails["Subject"].to_list()[0:10000])


CPU times: total: 54 s
Wall time: 54.1 s


In [5]:
N = len(prediction)
all_labels = [l["label"] for l in prediction[0]]
scores = {k: [] for k in all_labels}
for p in prediction[0:N]:
    for lsc in p:
        label = lsc["label"]
        score = lsc["score"]
        scores[label].append(score)
scores["subject"] = all_emails["Subject"].to_list()[0:N]
scores_df = pd.DataFrame(scores)


In [6]:
scores_df.head(20)


,joy,fear,anger,sadness,surprise,love,subject
0,0.684249,0.233480,0.052671,0.018999,0.005682,0.004919,"December 14, 2000 - Bear Stearns' predictions ..."
1,0.338514,0.327218,0.167740,0.133805,0.018015,0.014709,Bloomberg Power Lines Report
2,0.519616,0.061016,0.369791,0.037801,0.006358,0.005418,Consolidated positions: Issues & To Do list
3,0.519616,0.061016,0.369791,0.037801,0.006358,0.005418,Consolidated positions: Issues & To Do list
4,0.203277,0.174097,0.473060,0.110962,0.018560,0.020045,
5,0.579750,0.074367,0.290624,0.040772,0.005845,0.008642,Re: 2001 Margin Plan
6,0.565267,0.079276,0.328721,0.016512,0.004545,0.005679,"Var, Reporting and Resources Meeting"
7,0.282373,0.128080,0.488698,0.082371,0.010492,0.007986,Westgate
8,0.507950,0.122945,0.334595,0.026368,0.003676,0.004465,Meeting re: Storage Strategies in the West
9,0.203277,0.174097,0.473060,0.110962,0.018560,0.020045,
